# LayerLens MNIST Demo

This notebook demonstrates how to use LayerLens to explain a CNN model trained on the MNIST dataset.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models

# Import LayerLens
import sys
sys.path.append('..')  # Add parent directory to path
import layerlens as ll
from layerlens.utils import load_sample_data

## 1. Load the MNIST dataset

In [ ]:
# Load MNIST data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize pixel values
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Reshape for CNN input (add channel dimension)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

print(f"Training data shape: {x_train.shape}")
print(f"Test data shape: {x_test.shape}")

## 2. Create a simple CNN model for MNIST

In [ ]:
# Build a simple CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), name='conv1'),
    layers.MaxPooling2D((2, 2), name='pool1'),
    layers.Conv2D(64, (3, 3), activation='relu', name='conv2'),
    layers.MaxPooling2D((2, 2), name='pool2'),
    layers.Conv2D(64, (3, 3), activation='relu', name='conv3'),
    layers.Flatten(name='flatten'),
    layers.Dense(64, activation='relu', name='dense1'),
    layers.Dense(10, activation='softmax', name='output')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display model summary
model.summary()

## 3. Train the model

In [ ]:
# Train the model (or load pre-trained weights if available)
try:
    model.load_weights('mnist_cnn.h5')
    print("Loaded pre-trained weights")
except:
    print("Training the model...")
    model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.2)
    model.save_weights('mnist_cnn.h5')

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

## 4. Use LayerLens to explain the model

In [ ]:
# Create a LayerLens explainer
explainer = ll.Explainer(model)

# Select a sample image to explain
sample_idx = 42
sample_image = x_test[sample_idx:sample_idx+1]
sample_label = np.argmax(y_test[sample_idx])

# Display the sample image
plt.figure(figsize=(3, 3))
plt.imshow(sample_image[0, :, :, 0], cmap='gray')
plt.title(f"Digit: {sample_label}")
plt.axis('off')
plt.show()

# Generate explanations
explanations = explainer.explain(sample_image)

print(f"Generated explanations for {len(explanations.layer_explanations)} layers")

## 5. Visualize layer activations

In [ ]:
# Extract layer outputs
from layerlens.core.layer_extractor import LayerExtractor

extractor = LayerExtractor(model)
layer_outputs = extractor.extract(sample_image)

# Visualize activations for the first convolutional layer
from layerlens.visualization.heatmap_generator import generate_heatmap

conv1_activations = layer_outputs['conv1']
conv1_heatmap = generate_heatmap(conv1_activations, 'conv1', sample_image[0], overlay=False)

# Display the heatmap
from plotly.offline import iplot
iplot(conv1_heatmap)

## 6. Build surrogate models for each layer

In [ ]:
# Build surrogate models
from layerlens.core.surrogate_builder import SurrogateBuilder

surrogate_builder = SurrogateBuilder(surrogate_type='tree')

# Get a larger sample for training surrogates
n_samples = 200
sample_indices = np.random.choice(len(x_test), n_samples, replace=False)
sample_data = x_test[sample_indices]

# Extract layer outputs for all samples
all_layer_outputs = extractor.extract(sample_data)

# Build surrogate for the first dense layer
dense1_outputs = all_layer_outputs['dense1']
dense1_surrogate = surrogate_builder.fit('dense1', sample_data.reshape(n_samples, -1), dense1_outputs)

# Evaluate the surrogate
from layerlens.utils.plot_utils import plot_surrogate_vs_original

# Predict with both the surrogate and the original layer
surrogate_preds = dense1_surrogate.predict(sample_data.reshape(n_samples, -1))
original_preds = all_layer_outputs['dense1']

# Plot the comparison
surrogate_vs_original_fig = plot_surrogate_vs_original(
    surrogate_preds, original_preds, 
    title="Dense Layer 1: Surrogate vs Original"
)
plt.show()

## 7. Analyze feature importance

In [ ]:
# Get feature importances from the surrogate model
if hasattr(dense1_surrogate, 'feature_importances_'):
    feature_importances = dense1_surrogate.feature_importances_
    
    # Plot top feature importances
    from layerlens.utils.plot_utils import plot_feature_importance
    
    importance_fig = plot_feature_importance(
        feature_importances, 
        feature_names=None,  # Use default feature names
        top_n=20  # Show top 20 features
    )
    plt.show()

## 8. Visualize the model graph

In [ ]:
# Visualize the model architecture
from layerlens.visualization.layer_graph import plot_layer_graph

# Create the graph visualization
layer_graph = plot_layer_graph(model, highlight_layers='dense1')

# Display the graph
iplot(layer_graph)

## 9. Use the dashboard for interactive exploration

In [ ]:
# Create an interactive dashboard (this will open a new browser tab)
from layerlens.visualization.dashboard import show_dashboard

# Uncomment to run the dashboard
# show_dashboard(explanations, port=8050)
print("To launch the dashboard, uncomment the line above")

## 10. Explore feature flow through the network

In [ ]:
# Visualize how features flow through the network
from layerlens.visualization.feature_flow import plot_feature_flow

# Create the feature flow visualization
feature_flow_fig = plot_feature_flow(explanations, sample_image)

# Display the visualization
iplot(feature_flow_fig)

## Conclusion

In this notebook, we've demonstrated how to use LayerLens to:

1. Extract layer activations from a CNN model
2. Build interpretable surrogate models for layers
3. Visualize activations and feature importances
4. Explore the model's structure and behavior

LayerLens provides a comprehensive toolkit for understanding how deep learning models make decisions by analyzing the behavior of individual layers.